In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score
import pickle
import pickle
import math
from rouge_score import rouge_scorer
import matplotlib.pyplot as plt


In [ ]:
# read pickle as dataframe
pickle_path = "../nikos/uncertainty/wandb/run-20250414_225134-35cwfjoe/files/train_generations.pkl"